In [51]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_a11236eafade435eba5503fee8699fee"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_ef86a9b0ebd948f1847ba6b35de3a57d"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_fc0bbfc6ef684ad990f316e7cf9d8c76"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_dd3e69f3b0804684812ee3aca209f800"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_2a17f7f524114b498efcdad7e662f64a"},{"name":"output2","type":0,"uri":"tmp_15e0e317673f47b6a4028c2c0e1c1d4e"},{"name":"output3","type":0,"uri":"tmp_90ecb24b9dd647878955f19154a8335b"},{"name":"output4","type":0,"uri":"tmp_095a47cda04f4b9ab36efbca6b50f3d4"}]'

# 自定义参数


In [52]:
import wfio
import pandas as pd
from ustciscrBDL_B import get_score

# 读取并返回dataframe对象
# :param as_spark: 为True返回pyspark.sql.DataFrame对象，为False返回pandas.DataFrame对象，默认为False
train_data_1 = wfio.read_dataframe(_INPUT2)
train_data_2 = wfio.read_dataframe(_INPUT1)

#两个数据集里的列名不一致，下面给改成一致
mapper={'xy':'hy','xydl':'hydl','xyml':'hyml','xyzl':'hyzl','zczby':'zczb','jdxz':'xzjd',
        'frsjh':'fdbrxmp','djkyrq':'djrq','cwrysjh':'bsrxmmp','cwryxm':'bsrxm'}
train_data_2.rename(columns=mapper,inplace=True)

#两个训练集合并
train_data=pd.concat([train_data_1,train_data_2],ignore_index=True,sort=False)
#pd.set_option('display.max_columns',114)

In [53]:
hyml_dict = {'采矿业': 0,
        '电力、热力、燃气及水生产和供应业': 1,
        '房地产业': 2,
        '公共管理、社会保障和社会组织': 3,
        '国际组织': 4,
        '建筑业': 5,
        '交通运输、仓储和邮政业': 6,
        '教育': 7,
        '金融业': 8,
        '居民服务、修理和其他服务业': 9,
        '科学研究和技术服务业': 10,
        '农、林、牧、渔业': 11,
        '批发和零售业': 12,
        '水利、环境和公共设施管理业': 13,
        '卫生和社会工作': 14,
        '文化、体育和娱乐业': 15,
        '信息传输、软件和信息技术服务业': 16,
        '制造业': 17,
        '住宿和餐饮业': 18,
        '租赁和商务服务业': 19}

def process_data(data_df):
    processed_data = pd.DataFrame(columns=['aver_rkse','kpjds','hyml','zljds','have_bsr','jxje','aver_kpse','have_003','abnor_rk'])
    
    rows = data_df.shape[0]#rows=20000
    
    for row in range(rows):
        zl_num=0
        kp_num=0
        rkse_sum=0
        have_bsr=1
        jxse_sum=0
        kpse_sum=0
        have_003=0
        abnor_rk=0
        for col in range(0,96,8):
            if data_df.iloc[row,col]!=0:
                jxje=data_df.iloc[row,col+1]
                jxse=jxje*data_df.iloc[row,col+2]#进项税额
                kpse=data_df.iloc[row,col+5]#开票税额
                rkse=data_df.iloc[row,col+7]#入库税额
                
                kp_num+=1
                
                rkse_sum+=rkse
                jxse_sum+=jxse
                kpse_sum+=kpse
                
                if data_df.iloc[row,col+4]-data_df.iloc[row,col]>0:
                    zl_num+=1
                    
                if data_df.iloc[row,col+2]==0.03:
                    have_003=1
                    ying_rk=kpse
                else:
                    ying_rk=kpse-jxse
                    
                    
                if abs(ying_rk-rkse)>10 and jxje>10000000:
                    abnor_rk=1
                    
                    
                    
                    
        if kp_num!=0:
            aver_rkse=rkse_sum/kp_num
            aver_jxse=jxse_sum/kp_num
            aver_kpse=kpse_sum/kp_num
        else:
            aver_rkse=0
            aver_jxse=0
            
        if data_df.iloc[row,96]=='':
            have_bsr=0
        temp=pd.DataFrame([[aver_rkse,kp_num,0,zl_num,have_bsr,jxse_sum,aver_kpse,have_003,abnor_rk]],
                          columns=['aver_rkse','kpjds','hyml','zljds','have_bsr','jxje','aver_kpse','have_003','abnor_rk'])
        
        processed_data=processed_data.append(temp,ignore_index=True)
    
    processed_data['hyml']=data_df['hyml']
    processed_data['hyml']=processed_data['hyml'].map(hyml_dict)
    
    return processed_data

In [54]:
import numpy as np

#训练集的税务数据（前96列）转化为float类型
for label,data in train_data.loc[:,'2017q1fphdsl':'2019q4rkse'].iteritems():
    train_data[label]=data.apply(lambda x: 0 if x=='' else x).astype('float')

#得到处理后的数据
processed_data=process_data(train_data)
processed_data.head()

,aver_rkse,kpjds,hyml,zljds,have_bsr,jxje,aver_kpse,have_003,abnor_rk
0,67627.8200,3,19,0,1,7.447754e+05,315886.276667,0,0
1,52368.5800,3,3,0,0,1.167727e+06,424991.966667,0,0
2,9544.2825,4,12,0,1,2.480586e+06,271792.775000,0,0
3,225113.8300,1,5,0,0,1.892283e+04,244036.660000,0,0
4,188986.4300,2,19,0,1,6.564729e+05,400326.525000,0,0


In [55]:
#得到标签
label_data=train_data['yc'].astype('int')

#将数据转化为numpy.array格式
label_data=np.array(label_data)
processed_np_data=np.array(processed_data)

# hy_one_hot=np.eye(20)[processed_np_data[:,2].astype(np.int32)]
processed_np_data=np.delete(processed_np_data,(2,3,4),1)
# processed_np_data=np.hstack((processed_np_data,hy_one_hot))

processed_np_data[:10,:]

array([[67627.81999999999, 3, 744775.36727, 315886.2766666667, 0, 0],
       [52368.579999999994, 3, 1167726.5952, 424991.9666666666, 0, 0],
       [9544.2825, 4, 2480585.6299, 271792.77499999997, 0, 0],
       [225113.83, 1, 18922.82733, 244036.66, 0, 0],
       [188986.43, 2, 656472.8514, 400326.525, 0, 0],
       [0.0, 0, 0.0, 400326.525, 0, 0],
       [0.0, 1, 142693.3123, 111258.03, 0, 0],
       [332776.478, 5, 1097864.25708, 552349.33, 0, 0],
       [16466.376666666667, 3, 540081.59991, 190605.02333333335, 0, 0],
       [143625.035, 6, 1393396.07535, 317392.16000000003, 0, 0]],
      dtype=object)

In [56]:
#训练模型

from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

regre_model=LogisticRegression(penalty='l1',solver='liblinear')
#regre_model=DecisionTreeClassifier()
regre_model.fit(processed_np_data,label_data)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
#处理测试集数据

test_data = wfio.read_dataframe(_INPUT4)#测试集

get_score.post_user_id('12')

#测试集的税务数据（前96列）转化为float类型
for label,data in test_data.loc[:,'2017q1fphdsl':'2019q4rkse'].iteritems():
    test_data[label]=data.apply(lambda x: 0 if (x=='') or (x=='""""""""""""""') else x).astype('float')

#得到处理后的数据
test_processed_data=process_data(test_data)
test_processed_data.head()

,aver_rkse,kpjds,hyml,zljds,have_bsr,jxje,aver_kpse,have_003,abnor_rk
0,0.000,2,5,0,0,2.421384e+07,463386.070,0,1
1,0.000,0,5,0,0,0.000000e+00,463386.070,0,0
2,159.695,2,19,0,1,1.298528e+07,337943.815,0,1
3,292619.595,2,17,0,0,2.404985e+05,404169.075,0,0
4,0.000,2,19,2,1,2.367903e+07,225384.350,0,1


In [58]:
test_np_data=np.array(test_processed_data)

# hy_one_hot=np.eye(20)[test_np_data[:,2].astype(np.int32)]
test_np_data=np.delete(test_np_data,(2,3,4),1)
# test_np_data=np.hstack((test_np_data,hy_one_hot))

test_np_data[:5,:]

array([[0.0, 2, 24213838.9845, 463386.06999999995, 0, 1],
       [0.0, 0, 0.0, 463386.06999999995, 0, 0],
       [159.695, 2, 12985275.9941, 337943.815, 0, 1],
       [292619.595, 2, 240498.54381, 404169.075, 0, 0],
       [0.0, 2, 23679030.8438, 225384.35, 0, 1]], dtype=object)

In [59]:
#预测测试集

predicted = regre_model.predict_proba(test_np_data)
predicted[0:10,:]

array([[0.00000000e+00, 1.00000000e+00],
       [9.38719708e-01, 6.12802922e-02],
       [1.11022302e-15, 1.00000000e+00],
       [1.00000000e+00, 7.42022125e-47],
       [0.00000000e+00, 1.00000000e+00],
       [6.08884054e-11, 1.00000000e+00],
       [9.99998093e-01, 1.90707198e-06],
       [6.99396097e-12, 1.00000000e+00],
       [1.00000000e+00, 6.89930467e-48],
       [1.00000000e+00, 5.64858651e-79]])

In [60]:

result_data=pd.DataFrame(columns=['zjnsrsbh','Probability'])
result_data['zjnsrsbh']=test_data['zjnsrsbh']
result_data['Probability']=pd.DataFrame(predicted[:,1])

#result_data['Probability']=result_data['Probability'].apply(f)

get_score.post_verify_data(result_data)

User-id:12
程序开始运行时间为：2019-12-22 14:33:38.160035
程序结束运行时间为：2019-12-22 14:34:00.318197
程序运行时间（去除打分耗时）为：22.158162
程序的准确率为：93.46422011
最终得分为：94.1177981
